<a href="https://colab.research.google.com/github/kameda-yoshinari/DataAlgo-T/blob/master/DataAlgo_T(005)_DFS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3.3. Depth-First Search

We learn how to visit all the vertices in an undirected and connected graph by way of depth-first search.

**Reminder**  
On github, rendering might not be in good shape.  
To see the expected layout, open this page in Google Colaboratory.
To run one specific code cell in colab, click the icon on the left part or just type Ctrl + Enter.  
  

#Preparation
Connect the Jupyter environment and invoke a runtime. Mount your Google Drive by the procedure below.  
Change directory to the mounted point and make it as the working folder.  
By then, files are preserved even after you terminate the runtime environment.

In [ ]:
!echo "Mounting yout Google Drive"
from google.colab import drive 
drive.mount('/content/drive')

In [ ]:
!echo "Make a working folder and chnage directory to it"
%cd /content/drive/My\ Drive
%mkdir -p DataAlgo/005
%cd       DataAlgo/005
!ls

Consider Graph G3.  
8 vertices, undirected, connected. 
It can be writen in C program by the form of adjacency matrix.
Element-1 means the edge exists betweeen row-ID-vertex and column-ID-vertex (they are adjacent).

![da2020-graph3](https://user-images.githubusercontent.com/45651568/84249701-1107b680-ab46-11ea-89b0-6a454ec80ec4.jpg)

In [ ]:
%%writefile graph3.h
// 8 vertices, undirected, no-weight.
#define N 8
int edge[N][N] = {
//   0 1 2 3 4 5 6 7
	{0,1,0,0,1,0,0,0}, // 0
	{1,0,0,0,0,0,1,0}, // 1
	{0,0,0,1,0,0,1,0}, // 2
	{0,0,1,0,0,0,0,1}, // 3
	{1,0,0,0,0,1,0,0}, // 4
 	{0,0,0,0,1,0,1,0}, // 5
	{0,1,1,0,0,1,0,1}, // 6
	{0,0,0,1,0,0,1,0}  // 7
};

Or you can change the program to feed your own graph.  
below is an example of a complete graph of 8 vertices.  
(Make sure which one you are going to use for compiling the program later)

In [ ]:
%%writefile graph3.h
// 8 vertices, undirected, no-weight.
#define N 8
int edge[N][N] = {
//   0 1 2 3 4 5 6 7
	{0,1,1,1,1,1,1,1}, // 0
	{1,0,1,1,1,1,1,1}, // 1
	{1,1,0,1,1,1,1,1}, // 2
	{1,1,1,0,1,1,1,1}, // 3
	{1,1,1,1,0,1,1,1}, // 4
	{1,1,1,1,1,0,1,1}, // 5
	{1,1,1,1,1,1,0,1}, // 6
	{1,1,1,1,1,1,1,0}, // 7
};

# Outline of DFS

In dungeon-type RPG like Rogue, it is a depth-first search to choose the room investigate from the room as far as possible as seen from the first room (start vertex). It can be said that it is a bold strategy.  

The vertices in the middle leading to the vertex are also listed in order. Here, all edges shall have the same length. 

In fact, we don't know which room is far away. So, when you choose the next room from one room, you just choose a room that you have not been to yet. 

Quickly invade the chosen room. In other words, the last room chosen is processed first. This is equivalent to Last-In-First-Out. 

Stack (First-In-Last-Out = Last-In-First-Out) is used to describe the algorithm of depth-first search. 

Suppose you come to visit a new room. 
The room number is listed as "visited" now.
Then, check the adjacent rooms and write down them on the stack (this action is called "push").
Then, pick one of the rooms you just wrote in the stack (this action is called "pop"), and go for visit, Do the same thing...

In some cases, you may come to a room where you can't find new rooms to go because every next room has already been visited.
In this case, you go back along the road that you came. 
Memo on the stack will tell you which room to go back. 

In some cases, a room on the stacktop might be the one you have visited. In such a case, just skip the vertex and make the stack lower.

For reference, when DFS was performed from vertex 0 for graph G3, the result of vertex enumeration by DFS was as follows. 

Here, when there are multiple vertices of the same priority, it is assumed that the vertex with the lower number is selected. 

DFS result: 0 1 6 2 3 7 5 4

# C program of DFS

**Purpose**

List up the vertices of a given graph in the order of depth-first search that utilizes stack.

**Explanation**

A starting vertex is specified at the beginning.  
Let's call the top of the stack stacktop.  
When you add a new object, put it on top of the stack.   
This action is called push.   
The action of removing the top object is called pop.   

**Program**

The function bfs() is void because it never fails.  
You do not need to move the data in the stack.
So you need only one variable to remember the top position in the stack.

Unlike queue, there is no listing result on stack area at the end of the process.  
We use visited[] to record the listing order. 

**Remarks**

Below is the figure of the stack in this program for Graph G3 starting at No.0.
When we visit the stack-top vertex, new candidates to visit next are pushd on the stack top (red numbers). 

Then, The stack top object is poped out (we remove it from the stack top) and we visit the room.

When visiting a vertex, if no new vertices are pushed, it means we choose a vertex that has been pushed previously.
Visiting Vertex 7 is the case. Since the Vertex 7 does not push any vertices, the next vertex to visit is Vertex 5, which actually has been pushed by Vertex 6 (3 times ago).




![005-dfs-C](https://user-images.githubusercontent.com/45651568/117356920-44d29400-aeef-11eb-965e-f55c5dd30f02.png)

In [ ]:
%%writefile dfs_simple_E.c
// Depth First Search.
// kameda[at]ccs.tsukuba.ac.jp, 2020.
#include <stdio.h> // printf()
#include "graph3.h"

// DFS main body
void dfs(int StartingRoom){
	int listed[N];  // 0 not yet, 1 listed
	int stack[N*N];
	int stacktop = 0;
    
    int CurrentRoom;
    int RoomToCheck;

	int visited[N]; // memory to record the listing order
	int num_visited = 0; // number of recorded vetices

	for (RoomToCheck = 0; RoomToCheck < N; RoomToCheck++)
		listed[RoomToCheck] = 0;
	
	printf("Start DFS from vertex %d.\n", StartingRoom);
    // Push StartingRoom to the stack
    stack[stacktop] = StartingRoom;
    stacktop++;

	// Keep working if the vertices are in the stack.
	while (stacktop > 0) {
		printf("Stack status: length %d, vertex at the stack top is  %d\n", stacktop, stack[stacktop - 1]);
		{int i; for (i = 0; i < stacktop; i++) printf("%d ", stack[i]); printf("\n");}

        // Investigate the vertex (taken from the stack top, removing it from the stack)
		printf("Take vertex %d on the stack top and start process (it is deleted from the stack).\n", stack[stacktop - 1]);
		CurrentRoom = stack[stacktop - 1];
        stacktop--;

		// Is this Vertex new to us?
		if (listed[CurrentRoom] == 0) {
			// Yes, so process this vertex.
			// Some extra process my be placed here.

			// It is recognized as being listed.
			printf("Vertex %d is treated as already listed from now.\n", CurrentRoom);
			listed[CurrentRoom] = 1;
			// Record the listing order.
			visited[num_visited] = CurrentRoom;
			num_visited++;

    		// CHoose next
			//   RoomToCheck is in descending order in order to follow the rule of 
			//  "smaller ID should be chosen if there are vertices at the same level"
			for (RoomToCheck = N-1; RoomToCheck >= 0; RoomToCheck--) {
   	    		if (edge[CurrentRoom][RoomToCheck] == 0) {
					printf("  (At Vertex %d) Vertex %d is not adjacent.\n", CurrentRoom, RoomToCheck);
				} else if (listed[RoomToCheck] != 0) {
					printf("  (At Vertex %d) Vertex %d is adjacent, but already listed before.\n", CurrentRoom, RoomToCheck);
        		} else {         
					printf("  (At Vertex %d) Vertex %d is adjacent, and not listed, so put it on the stack top.\n", CurrentRoom, RoomToCheck);
               		// Push RoomToCheck to the stack
					stack[stacktop] = RoomToCheck;
               		stacktop++;
				}
			}
		}
	}
	printf("No vertices left in the stack. Quiting.\n");

	printf("Result of List up by DFS.\n");
	for (CurrentRoom = 0; CurrentRoom < num_visited; CurrentRoom++)
		printf("%d ", visited[CurrentRoom]);
    printf("\n");

	return;
}

// Main function
int main(int argc, char *argv[]){
	dfs(0); // Call dfs() with the starting vertex ID
	return 0;
}

Compile it and check no errors.

In [ ]:
!gcc -Wall -o dfs_simple_E dfs_simple_E.c

Run it.

In [ ]:
!./dfs_simple_E

# Command line option to set starting vertex

**Purpose**

Let the program take the command line option as the ID of the starting vertex to list up vertices by BFS.

**Explanation**

Use atoi() to convert string at command line option to integer.
(It is desirable to use strtol() since it can handle errors while atoi() cannot do it.)

**Program**

Command line option is processed within main().

**Remarks**

No changes at dfs() function.  
As this is the second time, you should rewrite the above program by youself by following the DFS version before you see the program below.



In [ ]:
%%writefile dfs_xtraCommandLine_E.c
// Depth First Search.
// kameda[at]ccs.tsukuba.ac.jp, 2020.
// -- Extra --
//    Accept command line option
#include <stdio.h> // printf()
#include <stdlib.h> // atoi()
#include "graph3.h"

// DFS main body
void dfs(int StartingRoom){
	int listed[N];  // 0 not yet, 1 listed
	int stack[N*N];
	int stacktop = 0;
    
    int CurrentRoom;
    int RoomToCheck;

	int visited[N]; // memory to record the listing order
	int num_visited = 0; // number of recorded vetices

	for (RoomToCheck = 0; RoomToCheck < N; RoomToCheck++)
		listed[RoomToCheck] = 0;
	
	printf("Start DFS from vertex %d.\n", StartingRoom);
    // Push StartingRoom to the stack
    stack[stacktop] = StartingRoom;
    stacktop++;

	// Keep working if the vertices are in the stack.
	while (stacktop > 0) {
		printf("Stack status: length %d, vertex at the stack top is  %d\n", stacktop, stack[stacktop - 1]);
		{int i; for (i = 0; i < stacktop; i++) printf("%d ", stack[i]); printf("\n");}

        // Investigate the vertex (taken from the stack top, removing it from the stack)
		printf("Take vertex %d on the stack top and start process (it is deleted from the stack).\n", stack[stacktop - 1]);
		CurrentRoom = stack[stacktop - 1];
        stacktop--;

		// Is this Vertex new to us?
		if (listed[CurrentRoom] == 0) {
			// Yes, so process this vertex.
			// Some extra process my be placed here.

			// It is recognized as being listed.
			printf("Vertex %d is treated as already listed from now.\n", CurrentRoom);
			listed[CurrentRoom] = 1;
			// Record the listing order.
			visited[num_visited] = CurrentRoom;
			num_visited++;

    		// Choose next
			//  RoomToCheck is in descending order in order to follow the rule of 
			//  "smaller ID should be chosen if there are vertices at the same level"
			for (RoomToCheck = N-1; RoomToCheck >= 0; RoomToCheck--) {
   	    		if (edge[CurrentRoom][RoomToCheck] == 0) {
					printf("  (At Vertex %d) Vertex %d is not adjacent.\n", CurrentRoom, RoomToCheck);
				} else if (listed[RoomToCheck] != 0) {
					printf("  (At Vertex %d) Vertex %d is adjacent, but already listed before.\n", CurrentRoom, RoomToCheck);
        		} else {         
					printf("  (At Vertex %d) Vertex %d is adjacent, and not listed, so put it on the stack top.\n", CurrentRoom, RoomToCheck);
               		// Push RoomToCheck to the stack
					stack[stacktop] = RoomToCheck;
               		stacktop++;
				}
			}
		}
	}
	printf("No vertices left in the stack. Quiting.\n");

	printf("Result of List up by DFS.\n");
	for (CurrentRoom = 0; CurrentRoom < num_visited; CurrentRoom++)
		printf("%d ", visited[CurrentRoom]);
    printf("\n");

	return;
}

// Main function
int main(int argc, char *argv[]){
    int startvertex = 0;
    if (argc == 1) {
        printf("Start vertex not specified, so set No.0 as start.\n");
        startvertex = 0;
    } else if (argc == 2) {
        startvertex = atoi(argv[1]);
        if (startvertex < 0 || startvertex >= N) {
            printf("Illegal ID specified, so quit.\n");
            return -1;
        }
    }

	dfs(startvertex);  // Call dfs() with the starting vertex ID 
	return 0;
}

Compile it and check no errors.

In [ ]:
!gcc -Wall -o dfs_xtraCommandLine_E dfs_xtraCommandLine_E.c

Run it.

In [ ]:
!./dfs_xtraCommandLine_E 0

# C program of DFS by function call

**Purpose**

push and pop by user defined functions.

**Explanation**

Write user defined functions in a different source file.  
It could be treated as an external library in the future.．

**Program**

Original source file is splitted into three parts.
* a header file of queue operation
* source file of user defined functions
* application source file of BFS

**Remarks**

Separate compilation method is applied.
．

In [ ]:
%%writefile StackLib_E.h
// Stack management
// kameda[at]ccs.tsukuba.ac.jp, 2020.

// Initialization of stack
int *initstack(int );

// Add to the stack
int stackpush(int );

// Delete from the stack
int stackpop(void);

// Show the status of the stack
int showstack(void);

// Show the whole status of the stack
int showallstack(void);


In [ ]:
%%writefile StackLib_E.c
// Stack management
// kameda[at]ccs.tsukuba.ac.jp, 2020.
#include <stdio.h> // printf()
#include <stdlib.h> // calloc()
#include "StackLib_E.h" // Prototype check

int *stack = NULL; // Stack mainbody (needs initialization)
int stacksize = 0; // Stack size
int stacktop = 0; // Stack top position

// Initialization of stack
int *initstack(int n) {
    stack = calloc(n, sizeof(*stack));
    if (stack == NULL) {
        printf("[Error] Cannot obtain stack memory for %d elements.\n", n);
    }
    stacksize = n;
    return stack;
}

// Add to the stack
int stackpush(int v) {
    if (stack == NULL) {
        printf("[Error] Stack is not ready.\n");
        return -1;
    } else if (stacktop >= stacksize) {
        printf("[Error] Running out of stack memory. Currently %d elements.\n", stacksize);
        return -2;
    }
    printf("--stack:push %d (stacktop=%d)\n", v, stacktop);
    stack[stacktop] = v;
    stacktop++;
    return 0;
}

// Delete from the stack
int stackpop(void) {
    int v;
    if (stack == NULL) {
        printf("[Error] Stack is not ready.\n");
        return -1;
    } else if (stacktop <= 0) {
        printf("[Report] No elements in the stack.\n");
        return -2;
    }
    printf("--stack:pop    (stacktop=%d)\n", stacktop);
    v = stack[stacktop - 1];
    stacktop--;
    printf("--stack:pop %2d (stacktop=%d)\n", v, stacktop);
    return v;
}

// Show the status of the stack
int showstack(void) {
    int i;

    if (stack == NULL) {
        printf("[Error] Stack is not ready.\n");
        return -1;
    }

    printf("showstack : stacksize = %d, stacktop = %d\n", stacksize, stacktop);
    printf("showstack : ");
    for (i = 0; i < stacktop; i++) {
        printf("%d ", stack[i]);
    }
    printf("\n");

    return stacktop;
}

// Swho the whole status of the stack
int showallstack(void) {
    int i;

    if (stack == NULL) {
        printf("[Error] Stack is not ready.\n");
        return -1;
    }

    printf("showallstack: stacksize = %d, stacktop = %d\n", stacksize, stacktop);
    printf("showallstack: ");
    for (i = 0; i < stacksize; i++) {
        printf("%d ", stack[i]);
    }
    printf("\n");

    return stacksize;
}




Compile only the source file of user defined functions. This is equivalent to create a new (user degined) library.

In [ ]:
!gcc -Wall -c StackLib_E.c

Rewrite the dfs() function to use external user-defined functions for handling the stack.
StackLib_E.h will be a help as a quick reference.


In [ ]:
%%writefile dfs_xtraFuncCalls_E.c
// Depth First Search.
// kameda[at]ccs.tsukuba.ac.jp, 2020.
// -- Extra --
//    Accept command line option
//    Call functions for stackpush and stackpop
#include <stdio.h> // printf()
#include <stdlib.h> // atoi()
#include "StackLib_E.h"
#include "graph3.h"

// DFS main body
void dfs(int StartingRoom){
	int listed[N];  // 0 not yet, 1 done
    
    int CurrentRoom;
    int RoomToCheck;

	int visited[N]; // memory to record the listing order
	int num_visited = 0; // number of recorded vetices

	for (RoomToCheck = 0; RoomToCheck < N; RoomToCheck++)
		listed[RoomToCheck] = 0;

    // Initialization of the stack
    initstack(N*N);
	
	printf("Start DFS from vertex %d.\n", StartingRoom);
    // Push StartingRoom to the stack
    stackpush(StartingRoom);

    // Stack status
    // Stack operation is made at while() , so show the status just before that
    showstack();

	// Keep working if the vertices are in the stack.
	while ((CurrentRoom = stackpop()) >= 0) {
        // Investigate the vertex (taken from the stack top, removing it from the stack)
		printf("Take vertex %d on the stack top and start process (it is deleted from the stack).\n", CurrentRoom);

		if (listed[CurrentRoom] == 0) {
			// Some extra process my be placed here.

			// It is recognized as being listed.	  
			printf("Vertex %d is treated as already listed from now.\n", CurrentRoom);
			listed[CurrentRoom] = 1;
			// Record the listing order.
			visited[num_visited] = CurrentRoom;
			num_visited++;

        	// Choose next
			//  RoomToCheck is in descending order in order to follow the rule of 
			//  "smaller ID should be chosen if there are vertices at the same level"
			for (RoomToCheck = N-1; RoomToCheck >= 0; RoomToCheck--) {
   	    		if (edge[CurrentRoom][RoomToCheck] == 0) {
					printf("  (At Vertex %d) Vertex %d is not adjacent.\n", CurrentRoom, RoomToCheck);
				} else if (listed[RoomToCheck] != 0) {
					printf("  (At Vertex %d) Vertex %d is adjacent, but already listed before.\n", CurrentRoom, RoomToCheck);
        		} else {         
					printf("  (At Vertex %d) Vertex %d is adjacent, and not listed, so put it on the stack top.\n", CurrentRoom, RoomToCheck);
               		// Push RoomToCheck to the stack
					stackpush(RoomToCheck);
				}
			}
		}
		
	    // Stack status
    	// Stack operation is made at while() , so show the status just before that
        showstack();
	}
	printf("No vertices left in the stack. Quiting.\n");

	printf("Result of List up by DFS.\n");
	for (CurrentRoom = 0; CurrentRoom < N; CurrentRoom++)
		printf("%d ", visited[CurrentRoom]);
    printf("\n");

	return ;
}

// Main function
int main(int argc, char *argv[]){
    int startvertex = 0;
    if (argc == 1) {
        printf("Start vertex not specified, so set No.0 as start.\n");
        startvertex = 0;
    } else if (argc == 2) {
        startvertex = atoi(argv[1]);
        if (startvertex < 0 || startvertex >= N) {
            printf("Illegal ID specified, so quit.\n");
            return -1;
        }
    }

	dfs(startvertex);  // Call dfs() with the starting vertex ID 
	return 0;
}

Compile the dfs program part.

In [ ]:
!gcc -Wall -c dfs_xtraFuncCalls_E.c

Link the object files (*.o) to make the binary file (dfs_xtraFuncCalls_E).

In [ ]:
!gcc -o dfs_xtraFuncCalls_E dfs_xtraFuncCalls_E.o StackLib_E.o

Run it. Output should be same as the previous one.

In [ ]:
!./dfs_xtraFuncCalls_E 0

# C program of DFS by recursive calls

**Purpose**

Write DFS program by using recursive function calls that are C language nature.


**Explanation**

Stack should be organized by the smart use of recursive function calls of C language.
On calling a function in C language, its arguments and local variables are stored in stack area.
When we call the same function for the several times, they are stacked (pushed) every time when the function is called.

This indicates the program shall be compact as we do not need to prepare stack operations by ourselves anymore.

**Program**

The use of listed[] and visited[] are same as before, but they are prepared as global variables because they are accessed at any function calls.
Initialization of the global variables are done at main function.
Result is also shown at the main function.

Note that the alogirhm is slightly different from the previous one because this program visits the new vertex while it is checking the adjacency of all the vertices in the graph (in other words, before checking ALL the adjacent vertices).

The stack diagram below is showing the relationship between the procedure and the stack area. Multiple "RoomToCheck and CurrentRoom" variable sets are stored (at most 4-layered; this means reccursive calls are 4 times nested). The arrow at RoomToCheck (X to Y) local variable indicates the loop (line 31-41) is going down from initial value X to value Y. If the Y comes to N (N=8 here), it means the recursive function call of that time ends.

**Remarks**

Note that the stack is changed in the different fashion as the previous program. Yet the result is the same.

![image](https://user-images.githubusercontent.com/45651568/90209118-75384680-de25-11ea-8b23-21203f576a68.png)
Figure: Recusive procedure diagram

In [ ]:
%%writefile dfs_recursive_E.c
// Depth First Search by recursive call
// kameda[at]ccs.tsukuba.ac.jp, 2020.
#include <stdio.h> // printf()
#include <stdlib.h> // atoi()
#include "graph3.h"

int listed[N];  // 0 not yet, 1 listed

int visited[N]; // memory to record the listing order
int num_visited = 0; // number of recorded vetices


// DFS main body by recursive calls
void dfs(int CurrentRoom){
    int RoomToCheck;

    // Start the process with something on the stack
	printf("Start process at this new Vertex %d.\n", CurrentRoom);

	// Some extra process my be placed here.

	// It is recognized as being listed.
	printf("Vertex %d is treated as already listed from now.\n", CurrentRoom);
	listed[CurrentRoom] = 1;
	// Record the listing order.
	visited[num_visited] = CurrentRoom;
	num_visited++;

    // Choose next
	for (RoomToCheck = 0; RoomToCheck < N; RoomToCheck++) {
   		if (edge[CurrentRoom][RoomToCheck] == 0) {
			printf("  (At Vertex %d) Vertex %d is not adjacent.\n", CurrentRoom, RoomToCheck);
		} else if (listed[RoomToCheck] != 0) {
			printf("  (At Vertex %d) Vertex %d is adjacent, but already listed before.\n", CurrentRoom, RoomToCheck);
        } else {         
			printf("  (At Vertex %d) Vertex %d is adjacent, and not listed, so visit that vetex right now!\n", CurrentRoom, RoomToCheck);
           	// Visit Vertex RoomToCheck (recursive call)
			dfs(RoomToCheck);
		}
	}

	return;
}

// Main function
int main(int argc, char *argv[]){
    int RoomToCheck;
    int startvertex = 0;
    if (argc == 1) {
        printf("Start vertex not specified, so set No.0 as start.\n");
        startvertex = 0;
    } else if (argc == 2) {
        startvertex = atoi(argv[1]);
        if (startvertex < 0 || startvertex >= N) {
            printf("Illegal ID specified, so quit.\n");
            return -1;
        }
    }

	for (RoomToCheck = 0; RoomToCheck < N; RoomToCheck++)
		listed[RoomToCheck] = 0;

	dfs(startvertex); // Call dfs() with the starting vertex ID

   	printf("Result of List up by DFS.\n");
	for (RoomToCheck = 0; RoomToCheck < N; RoomToCheck++)
		printf("%d ", visited[RoomToCheck]);
    printf("\n");

	return 0;
}

Compile it and check no errors.

In [ ]:
!gcc -Wall -o dfs_recursive_E dfs_recursive_E.c

Run it.

In [ ]:
!./dfs_recursive_E

# Short C program of DFS by recursive call

In the previous program, many printf() are inserted to show the flow of the procedure.

In the next program, dfs() consists of only the necessary statements. 
Exept for recursive call(line 22), there only one substitution statement (line 15), one printf() (line 17), one for() (line20), and one if() (line 21).

This is quite a good example of sophisticated use of built-in stack structure for DFS algorithm.

In [ ]:
%%writefile dfs_recursive-short_E.c
// Depth First Search by recursive call
// kameda[at]ccs.tsukuba.ac.jp, 2020.
#include <stdio.h> // printf()
#include <stdlib.h> // atoi()
#include "graph3.h"

int listed[N];  // 0 not yet, 1 listed

// DFS main body by recursive calls
void dfs(int CurrentRoom){
    int RoomToCheck;

	// It is recognized as being listed.
	listed[CurrentRoom] = 1;
	// Show the listing order.
    printf("Listing: %d\n", CurrentRoom);

    // Choose next
	for (RoomToCheck = 0; RoomToCheck < N; RoomToCheck++) {
        if (edge[CurrentRoom][RoomToCheck] != 0 && listed[RoomToCheck] == 0) 
			dfs(RoomToCheck);
	}
	return;
}

// Main function
int main(int argc, char *argv[]){
    int RoomToCheck;
    int startvertex = 0;
    if (argc == 1) {
        printf("Start vertex not specified, so set No.0 as start.\n");
        startvertex = 0;
    } else if (argc == 2) {
        startvertex = atoi(argv[1]);
        if (startvertex < 0 || startvertex >= N) {
            printf("Illegal ID specified, so quit.\n");
            return -1;
        }
    }

	for (RoomToCheck = 0; RoomToCheck < N; RoomToCheck++)
		listed[RoomToCheck] = 0;

	dfs(startvertex); // Call dfs() with the starting vertex ID

	return 0;
}

In [ ]:
!gcc -Wall -o dfs_recursive-short_E dfs_recursive-short_E.c

In [ ]:
!./dfs_recursive-short_E 0

# Alternative DFS algorithm (without recursive calls)

**Purpose**

Write the program that works same as the recursive-call program without using the recursive calls.
See the difference.

**Explanation**

Since we do not use recursive calls, we need to prepare stacks by ourselves.

**Program**

A mixture of dfs_xtraCommandLine_E.c and dfs_recursive_E.c.  
Make sure where is the part that works as recursive call.  
A structure variable is introduced to support CurrentRoom and RoomToCheck as stack.

**Remarks**

See how long (and complicated) the code is if we do not use recursive calls.

In [ ]:
%%writefile dfs_NoRecursive_E.c
// Depth First Search.
// kameda[at]ccs.tsukuba.ac.jp, 2021.
// -- Extra --
//    Accept command line option
//    No recursive calls
#include <stdio.h> // printf()
#include <stdlib.h> // atoi()
#include "graph3.h"

// A stack element
struct stackelement {
    int CurrentRoom;
    int RoomToCheck;
};

// DFS main body
void dfs(int StartingRoom){
	int listed[N];  // 0 not yet, 1 listed
    struct stackelement stack[N];
	int stacktop = 0;

	int visited[N]; // memory to record the listing order
	int num_visited = 0; // number of recorded vetices
    
    struct stackelement visiting; // local use (CurrentRoom and RoomToCheck)
    int r;

	for (r = 0; r < N; r++)
		listed[r] = 0;
	
	printf("Start DFS from vertex %d.\n", StartingRoom);
    // Push StartingRoom to the stack
    visiting.CurrentRoom = StartingRoom;
    visiting.RoomToCheck = 0;

	// Keep working if the vertices are in the stack.
	do {
		printf("Stack status: length %d, vertex at the stack top is  %d\n", stacktop, stack[stacktop - 1].CurrentRoom);
		{int i; printf("stack:RoomToCheck: "); for (i = 0; i < stacktop; i++) printf("%d ", stack[i].RoomToCheck); printf("\n");}
		{int i; printf("stack:CurrentRoom: "); for (i = 0; i < stacktop; i++) printf("%d ", stack[i].CurrentRoom); printf("\n");}

        // This part should be explicitly done only when it is new
        if (listed[visiting.CurrentRoom] == 0) {
    		// It is recognized as being listed.
	    	listed[visiting.CurrentRoom] = 1;
            // Show the listing (visiting) order
		    printf("Listing (Visiting): %d\n", visiting.CurrentRoom);
		    // Record the order just for the final report
		    visited[num_visited] = visiting.CurrentRoom;
		    num_visited++;
        }

   		// Choose next
		//  "smaller ID should be chosen if there are vertices at the same level"
		for ( ; visiting.RoomToCheck < N; visiting.RoomToCheck++) {
    		if (edge[visiting.CurrentRoom][visiting.RoomToCheck] == 0) {
				printf("  (At Vertex %d) Vertex %d is not adjacent.\n", visiting.CurrentRoom, visiting.RoomToCheck);
			} else if (listed[visiting.RoomToCheck] != 0) {
				printf("  (At Vertex %d) Vertex %d is adjacent, but already listed before.\n", visiting.CurrentRoom, visiting.RoomToCheck);
       		} else {         
				printf("  (At Vertex %d) Vertex %d is adjacent, and not listed, so put the current room %d on the stack top and go to the new room.\n", visiting.CurrentRoom, visiting.RoomToCheck, visiting.CurrentRoom);
        		// Push CurrentRoom/RoomToCheck to the stack for future use
				stack[stacktop] = visiting;
          		stacktop++;
                // And set new values to visit the new room
                visiting.CurrentRoom = visiting.RoomToCheck;
                visiting.RoomToCheck = 0;
                break; // go to the new room (by breaking the for loop)
			}
		}

        // End of CurrentRoom visiting?
        if (visiting.RoomToCheck >= N) {
     		printf("Remove (pop-up) an element on the stack top and back to vertex No. %d.\n", stack[stacktop - 1].CurrentRoom);
	    	visiting = stack[stacktop - 1];
            stacktop--;
        }
	} while (stacktop > 0) ;
	printf("No vertices left in the stack. Quiting.\n");

	printf("Result of List up by DFS.\n");
	for (r = 0; r < num_visited; r++)
		printf("%d ", visited[r]);
    printf("\n");

	return;
}

// Main function
int main(int argc, char *argv[]){
    int startvertex = 0;
    if (argc == 1) {
        printf("Start vertex not specified, so set No.0 as start.\n");
        startvertex = 0;
    } else if (argc == 2) {
        startvertex = atoi(argv[1]);
        if (startvertex < 0 || startvertex >= N) {
            printf("Illegal ID specified, so quit.\n");
            return -1;
        }
    }

	dfs(startvertex);  // Call dfs() with the starting vertex ID 
	return 0;
}

In [ ]:
!gcc -Wall -o dfs_NoRecursive_E dfs_NoRecursive_E.c

In [ ]:
!./dfs_NoRecursive_E 0

# Problems

1. Description of the algorithm   
Describe the breadth-first search algorithm using queues in Human language.   
The description must follow be the explanation in  this lecture.    
The diagram of the stack should be inserted. 


2. Complete Algorithm  
As long as a given graph is undirected and connected, the breadth-first search algorithm shown in this section always list up all vertices successfully. Try to explain that this is true. 


3. Computation amount   
Discuss the computation amount of time and space of the program in dfs_xtraCommandLine_E.c. 


4. Memory size of stack area  
No matter what undirected and connected graph is specified, show that the memory size required for stack does not exceed the number of vertices in dfs-alt_xtraFuncCalls_E.c program. 


5. Largest stack   
Find out the shape of a graph that gives the largest stack tentatively on processing the graph in dfs-alt_xtraFuncCalls_E.c program.


6. Simpler program by recursive call  
Why is the dfs_recursive_E.c program much simpler than dfs_simple_E.c program?
Try to explain the reason by refering C language structure.


7. Description of Recursive procedure  
Write down the document to understand the recursive prodedure diagram.




8. Insertion of extra phrase  
If you are asked to insert the statement of  
printf("  (At Vertex %d) Nothing to do here, so we are evacuated from this room.\n", CurrentRoom);  
in dfs_recursive_E.c, whch line should be the best place?


9. Distance display  
Modify the dfs_recursive-short_E.c program to show the distance from the starting vertex for each vertex on listing result, like:  
Start vertex not specified, so set No.0 as start.  
Listing: 0 (distance 0)  
Listing: 1 (distance 1)  
Listing: 6 (distance 2)  
Listing: 2 (distance 3)  
Listing: 3 (distance 4)  
Listing: 7 (distance 5)  
Listing: 5 (distance 3)  
Listing: 4 (distance 4)  





#**Course Info**

https://github.com/kameda-yoshinari/DataAlgo-T  
Course: Data structure and algorithm  
Department of Engineering Systems, University of Tsukuba,Japan.  
Author: KAMEDA, Yoshinari  
2021.05.07. revised  
2020.05.19. -  
